In [ ]:
import numpy as np
import json

In [ ]:
# Layer Class
class Layer:
    def __init__(self, neuron: int, activation: str, weights: np.array, bias: np.array):
        self.neuron = neuron
        self.weights = weights
        self.bias = bias
        if activation == 'linear':
            self.function = lambda x: x
        elif activation == 'relu':
            self.function = lambda x: np.maximum(0, x)
        elif activation == 'sigmoid':
            self.function = lambda x: 1 / (1 + np.exp(-x))
        elif activation == 'softmax':
            self.function = lambda x: np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
        else:
            raise ValueError('Invalid activation function')

    def forward(self, input: np.array):
        return self.function(np.dot(input, self.weights) + self.bias)

In [ ]:
class FFNN:
    def __init__(self, input: np.array, layers: list):
        self.input = input
        self.layers = layers

    def add_layer(self, layer: Layer):
        self.layers.append(layer)

    def forward(self):
        self.output = self.input
        for layer in self.layers:
            self.output = layer.forward(self.output)
        return self.output